In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [10]:
spark=SparkSession.builder.appName("Assignment").getOrCreate()

In [11]:
data=[{"Gender": "Male", "HeightCm": 171, "WeightKg": 96 },
{ "Gender": "Male", "HeightCm": 161, "WeightKg": 85 },
{ "Gender": "Male", "HeightCm": 180, "WeightKg": 77 },
{ "Gender": "Female", "HeightCm": 166, "WeightKg": 62},
{"Gender": "Female", "HeightCm": 150, "WeightKg": 70},
{"Gender": "Female", "HeightCm": 167, "WeightKg": 82}]

In [12]:
df=spark.createDataFrame(data)

C:\Apps\spark-3.0.0-bin-hadoop2.7\python\pyspark\sql\session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [13]:
df.show(6,0)

+------+--------+--------+
|Gender|HeightCm|WeightKg|
+------+--------+--------+
|Male  |171     |96      |
|Male  |161     |85      |
|Male  |180     |77      |
|Female|166     |62      |
|Female|150     |70      |
|Female|167     |82      |
+------+--------+--------+



##### 1) Calculate the BMI (Body Mass Index) using Formula 1, BMI Category and Health risk from Table 1 of the person and add them as 3 new columns
Formula: BMI(kg/m2) = mass(kg) / height(m)2

In [55]:
df_bmi=df.withColumn("BMI(kg/m2)", df['WeightKg']/F.pow((df['HeightCm']/100), 2))
df_bmi.show(6,0)

+------+--------+--------+------------------+
|Gender|HeightCm|WeightKg|BMI(kg/m2)        |
+------+--------+--------+------------------+
|Male  |171     |96      |32.83061454806607 |
|Male  |161     |85      |32.79194475521777 |
|Male  |180     |77      |23.76543209876543 |
|Female|166     |62      |22.49963710262738 |
|Female|150     |70      |31.11111111111111 |
|Female|167     |82      |29.402273297715947|
+------+--------+--------+------------------+



In [87]:
df2=df_bmi.withColumn("MI_Category", F.when(df_bmi['bmi(kg/m2)']<=18.4, 'Underweight').when((df_bmi['bmi(kg/m2)']>=18.5)&(df_bmi['bmi(kg/m2)']<24.9), 'Normal weight')\
                  .when((df_bmi['bmi(kg/m2)']>=25)&(df_bmi['bmi(kg/m2)']<29.9), 'Overweight')\
                  .when((df_bmi['bmi(kg/m2)']>=30)&(df_bmi['bmi(kg/m2)']<34.9), 'Moderately obese')\
                  .when((df_bmi['bmi(kg/m2)']>=35)&(df_bmi['bmi(kg/m2)']<39.9), 'Severely obese')\
                  .otherwise(F.lit('Very severely obese')))
df2.show(6,0)


+------+--------+--------+------------------+----------------+
|Gender|HeightCm|WeightKg|BMI(kg/m2)        |MI_Category     |
+------+--------+--------+------------------+----------------+
|Male  |171     |96      |32.83061454806607 |Moderately obese|
|Male  |161     |85      |32.79194475521777 |Moderately obese|
|Male  |180     |77      |23.76543209876543 |Normal weight   |
|Female|166     |62      |22.49963710262738 |Normal weight   |
|Female|150     |70      |31.11111111111111 |Moderately obese|
|Female|167     |82      |29.402273297715947|Overweight      |
+------+--------+--------+------------------+----------------+



In [95]:
df3=df2.withColumn('Health risk', F.when(df2['MI_Category']=='Underweight', 'Malnutrition risk')\
               .when(df2['MI_Category']=='Normal weight', 'Low risk')\
               .when(df2['MI_Category']=='Overweight', 'Enhanced risk')\
               .when(df2['MI_Category']=='Moderately obese', 'Medium risk')\
               .when(df2['MI_Category']=='Severely obese', 'High risk')\
               .otherwise('Very high risk'))
df3.show(6,0)

+------+--------+--------+------------------+----------------+-------------+
|Gender|HeightCm|WeightKg|BMI(kg/m2)        |MI_Category     |Health risk  |
+------+--------+--------+------------------+----------------+-------------+
|Male  |171     |96      |32.83061454806607 |Moderately obese|Medium risk  |
|Male  |161     |85      |32.79194475521777 |Moderately obese|Medium risk  |
|Male  |180     |77      |23.76543209876543 |Normal weight   |Low risk     |
|Female|166     |62      |22.49963710262738 |Normal weight   |Low risk     |
|Female|150     |70      |31.11111111111111 |Moderately obese|Medium risk  |
|Female|167     |82      |29.402273297715947|Overweight      |Enhanced risk|
+------+--------+--------+------------------+----------------+-------------+



##### 2)Count the total number of overweight people

In [97]:
df3.filter(df3['MI_Category']=='Overweight').count()

1